In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/adausdt.csv")
df.head()

,Opentime,Open,High,Low,Close,Volume,Closetime,Quote asset volume,Number of trades,Taker by base,Taker buy quote,Ignore
0,2021-11-01 08:00:00,1.965,2.019,1.911,1.949,144061135.4,1635811199999,2.824457e+08,443944,71078236.0,1.394258e+08,0
1,2021-11-02 08:00:00,1.949,1.996,1.933,1.971,124263204.0,1635897599999,2.446304e+08,382592,64339347.3,1.267002e+08,0
2,2021-11-03 08:00:00,1.971,2.144,1.950,2.063,335561983.7,1635983999999,6.936646e+08,1237122,167743048.3,3.467647e+08,0
3,2021-11-04 08:00:00,2.063,2.103,1.933,1.983,174549100.4,1636070399999,3.509825e+08,586577,80789835.4,1.624612e+08,0
4,2021-11-05 08:00:00,1.983,2.017,1.959,1.982,108180863.7,1636156799999,2.150281e+08,346939,51873631.3,1.030921e+08,0


In [3]:
import sys
sys.path.append('../')
from libbot import *

## Result

In [4]:
ewma_slow = df['Close'].ewm(span=26, adjust=False).mean()
ewma_fast = df['Close'].ewm(span=12, adjust=False).mean()

In [5]:
macd_tb = compute_macd(df['Close'], 26, 12, 9)
macd_tb.tail()

,macd,signal,hist
269,0.006639,0.001747,0.004891
270,0.008627,0.003123,0.005503
271,0.010445,0.004588,0.005857
272,0.010937,0.005857,0.005079
273,0.010902,0.006866,0.004036


In [6]:
close_prices = df['Close']
buy_price, sell_price, macd_signal = macd_strategy(close_prices, macd_tb)

In [7]:
macd_tb['position'] = create_position(close_prices, macd_signal)
macd_tb['macd_signal'] = macd_signal
macd_tb.tail()

,macd,signal,hist,position,macd_signal
269,0.006639,0.001747,0.004891,1,0
270,0.008627,0.003123,0.005503,1,0
271,0.010445,0.004588,0.005857,1,0
272,0.010937,0.005857,0.005079,1,0
273,0.010902,0.006866,0.004036,1,0


In [8]:
cp_returns = np.diff(close_prices)

In [9]:
from math import floor

investment_value = 1e5
num_of_stocks = floor(investment_value / close_prices[0])
num_of_stocks

51308

In [10]:
strategy_return = []
# cp_returns * macd_tb['position'][:-1]
for i in range(1,len(cp_returns)):
    r = cp_returns[i] * macd_tb['position'][i]
    strategy_return.append(r)

strategy_return

[0.09200000000000008,
 -0.08000000000000007,
 -0.001000000000000112,
 0.02299999999999991,
 0.016000000000000014,
 0.10400000000000009,
 0.14500000000000002,
 -0.17300000000000004,
 -0.017999999999999794,
 -0.03400000000000025,
 0.004999999999999893,
 -0.009999999999999787,
 -0.02300000000000013,
 -0.1429999999999998,
 0.0019999999999997797,
 -0.08999999999999986,
 0.07699999999999996,
 0.05899999999999994,
 -0.08699999999999997,
 -0.05899999999999994,
 -0.025000000000000133,
 -0.08499999999999996,
 0.009000000000000119,
 -0.14200000000000013,
 0.010000000000000009,
 0.051000000000000156,
 0.006999999999999895,
 -0.04699999999999993,
 -0.006000000000000005,
 0.17100000000000004,
 -0.16300000000000003,
 -0.13600000000000012,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.07099999999999995,
 -0.02200000000000002,
 0.02400000000000002,
 0.001000000000000112,
 -0.006000000000000005,
 0.04299999999999993,
 0.04699999999999993,
 0.14800000000000013,
 -0.083000000

In [11]:
np.sum(strategy_return)

-0.966

## RSI strategy

In [12]:
df = pd.read_csv("../data/adausdt.csv")
df.head()

,Opentime,Open,High,Low,Close,Volume,Closetime,Quote asset volume,Number of trades,Taker by base,Taker buy quote,Ignore
0,2021-11-01 08:00:00,1.965,2.019,1.911,1.949,144061135.4,1635811199999,2.824457e+08,443944,71078236.0,1.394258e+08,0
1,2021-11-02 08:00:00,1.949,1.996,1.933,1.971,124263204.0,1635897599999,2.446304e+08,382592,64339347.3,1.267002e+08,0
2,2021-11-03 08:00:00,1.971,2.144,1.950,2.063,335561983.7,1635983999999,6.936646e+08,1237122,167743048.3,3.467647e+08,0
3,2021-11-04 08:00:00,2.063,2.103,1.933,1.983,174549100.4,1636070399999,3.509825e+08,586577,80789835.4,1.624612e+08,0
4,2021-11-05 08:00:00,1.983,2.017,1.959,1.982,108180863.7,1636156799999,2.150281e+08,346939,51873631.3,1.030921e+08,0


In [13]:
rets = df['Close'].diff()
rets.head()

0      NaN
1    0.022
2    0.092
3   -0.080
4   -0.001
Name: Close, dtype: float64

In [14]:
rsi_df = pd.DataFrame(index=df.index)

rsi_df['up'] = np.where(rets < 0, rets, 0)
rsi_df['down'] = np.where(rets > 0, rets, 0)

In [15]:
rsi_df.head()

,up,down
0,0.000,0.000
1,0.000,0.022
2,0.000,0.092
3,-0.080,0.000
4,-0.001,0.000
